In [1]:
import lyricsgenius
from multiprocessing import current_process, Pool
import time
import sqlite3

In [2]:
genius_tokens = [
    "tETKVL7G-wtix9PAAPxsDeW-j8sS1D4L-zyWei7n1Xq1WUh9VuwFLHAXlBP5X5zq",
    "6rUGP-maK090aBdaNi1Z0qm2CtFDoIyteABaTh441aYc4eKHyKYPv5Q8luozmFC0",
    "whDHSeFHAeKA9f3mEt63XMBJVBBp-OO-01qRzn4mu8L1pQoI5hFywcm5MK2DkxgV",
    "Cj82uimfARNlK36SjqkeXAqpqoWdACXWwm1kIRHzYLAbkUiHYcvNqfj9Iq99J8iR"
]

In [3]:
genius_apis = []
for token in genius_tokens:
    genius_apis.append(lyricsgenius.Genius(token))

In [4]:
def get_lyrics(song, artist):
    proc = current_process()._identity[0]
    try:
        return genius_apis[proc%4].search_song(song, artist)
    except:
        return get_lyrics(song, artist)

In [5]:
song_names = [
    ["The Magician", "Andy Shauf"],
    ["Begin Again", "Andy Shauf"],
    ["Martha Sways", "Andy Shauf"],
    ["I Will", "The Beatles"],
    ["90210", "Travis Scott"],
    ["Soma", "The Strokes"],
    ["Hey Jude", "The Beatles"],
    ["Hey Bulldog", "The Beatles"],
    ["Self Care", "Mac Miller"],
    ["Machu Picchu", "The Strokes"],
    ["Blue Boy", "Mac DeMarco"],
    ["Dreams Via Memories", "Ceramic Animal"],
    ["White Gloves", "Khruangbin"],
    ["Down the Line", "Beach Fossils"],
    ["Sleep Apnea", "Beach Fossils"]
]

In [ ]:
songs = []
start = time.perf_counter()
for song_name in song_names:
    songs.append(get_lyrics(song_name[0], song_name[1]))
print(songs)
print(time.perf_counter() - start, " seconds for ", len(song_names), " songs")

In [6]:
start = time.perf_counter()
with Pool(processes=4) as p:
    songs = p.starmap(get_lyrics, song_names)
print(songs)
print(time.perf_counter() - start, " seconds for ", len(song_names), " songs")

Searching for "The Magician" by Andy Shauf...Searching for "Martha Sways" by Andy Shauf...

Searching for "Begin Again" by Andy Shauf...
Searching for "I Will" by The Beatles...
Done.
Searching for "90210" by Travis Scott...Done.Done.
Searching for "Soma" by The Strokes...


Done.Searching for "Hey Jude" by The Beatles...

Searching for "Hey Bulldog" by The Beatles...
Done.
Searching for "Self Care" by Mac Miller...
Done.
Searching for "Machu Picchu" by The Strokes...
Done.Done.

Searching for "Blue Boy" by Mac DeMarco...Searching for "Dreams Via Memories" by Ceramic Animal...

Done.
Searching for "White Gloves" by Khruangbin...
Done.
Searching for "Down the Line" by Beach Fossils...
Done.
Searching for "Sleep Apnea" by Beach Fossils...
Done.
Done.
Done.
Done.
[('The Magician', 'Andy Shauf'), ('Begin Again', 'Andy Shauf'), ('Martha Sways', 'Andy Shauf'), ('I Will', 'The Beatles'), ('90210', 'Travis Scott'), ('Soma', 'The Strokes'), ('Hey Jude', 'The Beatles'), ('Hey Bulldog', 'The Beat

In [ ]:
for song in songs:
    print(song.lyrics)